In [ ]:
%pip install -q ipyleaflet==0.17.3 ipywidgets==8.0.6 jupyterlab_widgets==3.0.7

# Emission Plumes

In [ ]:
import json
from js import fetch

import pandas as pd
import geopandas as gpd
import ipywidgets as widgets

from ipyleaflet import Map, TileLayer, GeoData, WidgetControl, LayersControl

In [ ]:
# Download using this URL, then upload to GIST for easy access
GDRIVE_ID = "1f9HCtAOUiSDTiesbJrt8E9Oi9wr6e8XT"
DOWNLOAD_URL = f"https://drive.google.com/uc?{GDRIVE_ID}=&export=download"

GIST_URL = "https://gist.githubusercontent.com/jsignell/53f6f211a17c2d01a8546e292d800c8a/raw/6de1c6555e9dfb5f127f81ff03603373c1fc52cd/gistfile1.txt"

async def fetch_json(url):
    response = await fetch(url)
    text = await response.text()
    return json.loads(text)
    return result

result = await fetch_json(GIST_URL)
df = gpd.GeoDataFrame.from_features(result["features"])

outlines = df[df.geom_type == "Polygon"].reset_index(drop=True)
centers = df[df.geom_type == "Point"].reset_index(drop=True)

In [ ]:
# Get the ids for every item in the STAC collection
STAC_SEARCH_URL = "https://dev.ghg.center/api/stac/search?collections=nasa-jpl-plumes-emissions-updated&fields=id,geometry,properties&limit=1000"

result = await fetch_json(STAC_SEARCH_URL)

In [ ]:
# Workaround for https://github.com/geopandas/geopandas/issues/1208
def move_id_into_props(f):
    f["properties"]["id"] = f["id"] 
    return f

result["features"] = list(map(move_id_into_props, result["features"]))
item_ids = gpd.GeoDataFrame.from_features(result["features"])

In [ ]:
IDS_ON_MAP = set()

m = Map(center=(0, 0), zoom=2, scroll_wheel_zoom=True)
m.layout.min_height="800px"

polygons = GeoData(
    geo_dataframe=outlines, 
    name='outlines',
    style={"color": "red"},
    hover_style={'fillColor': 'red', "fillOpacity": 0.5},
)

pins = GeoData(
    geo_dataframe=centers, 
    name='centers',
)

out = widgets.Output(layout=widgets.Layout(width="800px"))

start_date = outlines["UTC Time Observed"].min().split("T")[0]
end_date = outlines["UTC Time Observed"].max().split("T")[0]

dates = [d.date().isoformat() for d in pd.date_range(start_date, end_date, freq='D')]
index = (0, len(dates)-1)

date_range = widgets.SelectionRangeSlider(
    options=dates,
    index=index,
    orientation='horizontal',
    layout={'width': '300px'},
    style={'description_width': 'initial'},
    readout=False
)

start_label = widgets.Label(date_range.value[0])
end_label = widgets.Label(date_range.value[1])

def filter_data(start, end):
    global outlines
    global centers
    
    centers_subset = centers[(centers["UTC Time Observed"] > start) & (centers["UTC Time Observed"] < end)]
    pins.data = json.loads(centers_subset.to_json())
    
    outlines_subset = outlines[(outlines["UTC Time Observed"] > start) & (outlines["UTC Time Observed"] < end)]
    polygons.data = json.loads(outlines_subset.to_json())

def callback(dts):
    start = dts[0]
    end = dts[1]
    start_label.value = start
    end_label.value = end
    filter_data(start, end)

widgets.interactive_output(
    callback, 
    {"dts": date_range}
)

date_range_widget = widgets.HBox([
    start_label,
    date_range,
    end_label,
])

m.add(polygons)
m.add(pins)
m.add(WidgetControl(widget=out, position="bottomleft"))
m.add(WidgetControl(widget=date_range_widget, position="topright"))

def display_properties(feature):
    out.clear_output()
    p = {k: v for k, v in feature["properties"].items() if k not in ["style"]}
    with out:
        display(pd.Series(p))

def add_raster(feature):
    global item_ids
    global m
    
    props = feature["properties"]

    collection = "nasa-jpl-plumes-emissions-updated"
    assets = "ch4-plume-emissions"
    
    if feature["id"] not in IDS_ON_MAP:
        subset = item_ids[item_ids.id.str.startswith(props["Scene FID"])]
        outline_shape = outlines.iloc[int(feature["id"])].geometry
        subset = subset[subset.intersects(outline_shape)]

        for item in subset.id:
            TILE_URL = (
                'https://dev.ghg.center/api/raster/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x'
                f'?collection={collection}&item={item}&assets={assets}'
                '&resampling=bilinear&bidx=1&colormap_name=plasma&rescale=1%2C1500&nodata=-9999'
            )
            m.add_layer(TileLayer(url=TILE_URL, max_zoom=24, show_loading=True))
        IDS_ON_MAP.add(feature["id"])
        
    m.center = (props['Latitude of max concentration'], props['Longitude of max concentration'])
    m.zoom = 12
    
def set_date_range(feature):
    global date_range
    
    props = feature["properties"]
    
    t = pd.Timestamp(props["UTC Time Observed"])
    date_range.value = ((t - pd.Timedelta("1D")).date().isoformat(), (t + pd.Timedelta("1D")).date().isoformat())
    

def on_click(event, feature, **kwargs):
    display_properties(feature)
    add_raster(feature)
    set_date_range(feature)

polygons.on_click(on_click)
pins.on_click(on_click)
m

⚠️ This visualization was created using a [jupyterlite-pyodide-kernel](https://github.com/jupyterlite/pyodide-kernel) and [voici](https://voici.readthedocs.io/en/latest/). It is rendered client-side (in your browser!) via the magic of [Wasm](https://webassembly.org/). This is an exciting and rapidly evolving space which means that this visualization is highly experimental and likely to break. ⚠️